In [1]:
from math import sqrt, cos, sin, pi

In [2]:
import numpy as np

In [3]:
from geometry import triangulate, transform, wall_pieces

In [4]:
from helpers import make_mesh, write_mesh

In [5]:
WALL = 0.4
FLOOR = 0.6

In [6]:
lat_n = 20
long_n = 10

In [7]:
inner_radius = 1
outer_radius = 1.5

In [8]:
surface = []

big_r = outer_radius + inner_radius
small_r = outer_radius - inner_radius

for j in range(0, long_n):
    tx, ty = cos(pi / 4 * j / long_n), sin(pi / 4 * j / long_n)
    sx, sy = cos(pi / 4 * (j + 1) / long_n), sin(pi / 4 * (j + 1) / long_n)
        
    surface.append([(tx * big_r, ty * big_r, 0), (sx * big_r, sy * big_r, 0), (big_r, big_r, 0)])
    surface.append([(tx * small_r, ty * small_r, 0), (sx * small_r, sy * small_r, 0), (0, 0, 0)])

    for i in range(0, lat_n):
        # x and z coordinates
        p1 = (outer_radius + cos(pi * i / lat_n) * inner_radius, -sin(pi * i / lat_n) * inner_radius)
        p2 = (outer_radius + cos(pi * (i + 1) / lat_n) * inner_radius, -sin(pi * (i + 1) / lat_n) * inner_radius)
        a1, a2 = (tx * p1[0], ty * p1[0], p1[1]), (tx * p2[0], ty * p2[0], p2[1])
        b1, b2 = (sx * p1[0], sy * p1[0], p1[1]), (sx * p2[0], sy * p2[0], p2[1])

        for tri in triangulate((a1, a2, b2, b1)):
            surface.append(tri)

In [9]:
surface = transform(surface, lambda x, y, z: (y, x, z + 1))

In [10]:
front = []
for i in range(0, lat_n):
    # x and z coordinates
    p1 = (outer_radius + cos(pi * i / lat_n) * inner_radius, -sin(pi * i / lat_n) * inner_radius)
    p2 = (outer_radius + cos(pi * (i + 1) / lat_n) * inner_radius, -sin(pi * (i + 1) / lat_n) * inner_radius)

    corner = (outer_radius + inner_radius, -inner_radius) if i < lat_n / 2 else (0, -inner_radius)
    tri = (p1, p2, corner)
    front.append([(p[0], 0, p[1]) for p in tri])

front.append([(0, 0, 0), (0, 0, -inner_radius), (outer_radius - inner_radius, 0, 0)]) 

In [11]:
front = transform(front, lambda x, y, z: (y, x, z + 1))

In [13]:
def make_5_piece(eighth, front_panel, wall, floor):
    quarter = transform(eighth, lambda x, y, z: (x, y, z + floor)) + transform(eighth, lambda x, y, z: (y, x, z + floor))

    width = max(p[0] for tri in eighth for p in tri)
    front_bottom_rect = [
        (0, 0, 0), (0, 0, floor), (0, width, floor), (0, width, 0)
    ]
    front = transform(front_panel, lambda x, y, z: (x, y, z + floor)) + list(triangulate(front_bottom_rect))

    mould_quarter = quarter + front

    shape_height = max(p[2] for tri in eighth for p in tri)
    height = shape_height + floor
    size = width + wall
    intermediate_heights = [p[2] for tri in front_panel for p in tri if p[1] == 0 and p[0] == width]
    heights = [0, floor] + [x + floor for x in intermediate_heights] + [height]

    rim_quarter = list(triangulate(
        [(0, width, height), (0, size, height), (size, size, height), (width, width, height)]
    ))

    top_rim = (
        rim_quarter
        + transform(rim_quarter, lambda x, y, z: (y, x, z))
        + transform(rim_quarter, lambda x, y, z: (x, -y, z))
        + transform(rim_quarter, lambda x, y, z: (y, -x, z))
    )

    floor_square = list(triangulate(
        [(0, 0, 0), (0, width, 0), (width, width, 0), (width, 0, 0)]
    ))
    floor_rim = transform(top_rim, lambda x, y, z: (x, y, 0))
    floor_all = floor_rim + floor_square

    left_outer_pieces = [
        [(0, width), (0, size)],
        [(0, size), (size, size)],
        [(size, size), (size, 0)],
    ]

    outer_pieces = left_outer_pieces + [[(x, -y) for x, y in piece] for piece in left_outer_pieces]
    wall_outer = list(tri for piece in outer_pieces for tri in wall_pieces(*piece, heights=heights))

    wrapper = floor_all + wall_outer + top_rim
    outer_mould = mould_quarter + wrapper
    division = (
        transform(front, lambda x, y, z: (y, x, z))
        + list(wall_pieces((width, 0), (width, -width), heights))
        + list(wall_pieces((width, -width), (0, -width), heights))
    )

    filler_square = transform(floor_square, lambda x, y, z: (x, -y, z))
    filler_mould_quarter = transform(mould_quarter, lambda x, y, z: (x, -y, z))
    filler = filler_square + filler_mould_quarter

    piece_1 = outer_mould + filler
    piece_4 = outer_mould + division
    piece_5 = filler + division
    
    return piece_1, piece_4, piece_5

In [22]:
pieces = make_5_piece(surface, front, WALL, FLOOR)

In [23]:
MM_SCALE = 15

In [24]:
def write_all_meshes(pieces, scale, name):
    piece_1, piece_4, piece_5 = pieces
    write_mesh(piece_1, scale, 'output/%s_1.stl' % (name, ))
    write_mesh(piece_4, scale, 'output/%s_4.stl' % (name, ))
    write_mesh(piece_5, scale, 'output/%s_5.stl' % (name, ))

In [25]:
write_all_meshes(pieces, MM_SCALE, "torus")